In [25]:
import os
os.getcwd()

'/workspace'

In [5]:
from __future__ import division
# Codes are borrowed from https://github.com/vikasverma1077/manifold_mixup/tree/master/supervised

import os, sys, shutil, time, random
from collections import OrderedDict

sys.path.append('..')
sys.path.append('/workspace/hi/DropMix/Mixup_imagenet_exp/')
if sys.version_info[0] < 3:
    import cPickle as pickle
else:
    import _pickle as pickle
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable
from load_data import load_data_subset
from logger import plotting, copy_script_to_folder, AverageMeter, RecorderMeter, time_string, convert_secs2time
import models
from multiprocessing import Pool

os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'

In [20]:
ngpu = 1
arch = 'wrn28_2'
batch_size = 128
dataset = 'cifar100'
stride = 1 # if cifar100
num_classes = 100
data_dir = ''
labels_per_class = 500
valid_labels_per_class = 500
dropout = False

In [24]:
resume = '/workspace/hi/DropMix/Mixup_imagenet_exp/experiments/cifar100_arch_wrn28_2_train_vanilla_eph_300_lr_0.1_transport_eps_0.8_size_-1_job_id__seed_0/model_best.pth.tar'

train_loader, valid_loader, _, test_loader, num_classes = load_data_subset(
    batch_size,
    2,
    dataset,
    data_dir,
    labels_per_class=labels_per_class,
    valid_labels_per_class=valid_labels_per_class,
    mixup_alpha=1.0)

# create model
print("=> creating model '{}'".format(arch))
net = models.__dict__[arch](num_classes, dropout, stride).cuda()
net = torch.nn.DataParallel(net, device_ids=list(range(ngpu)))

if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    recorder = checkpoint['recorder']
    start_epoch = checkpoint['epoch']
    net.load_state_dict(checkpoint['state_dict'])
    best_acc = recorder.max_accuracy(False)
    print(
        "=> loaded checkpoint '{}' accuracy={} (epoch {})".format(
            resume, best_acc, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))


FileNotFoundError: [Errno 2] No such file or directory: ''

In [15]:
checkpoint.keys()

dict_keys(['epoch', 'arch', 'state_dict', 'recorder', 'optimizer'])

In [ ]:
global best_acc

# dataloader
train_loader, valid_loader, _, test_loader, num_classes = load_data_subset(
    args.batch_size,
    2,
    args.dataset,
    args.data_dir,
    labels_per_class=labels_per_class,
    valid_labels_per_class=valid_labels_per_class,
    mixup_alpha=1.0)

# create model
print("=> creating model '{}'".format(arch))
net = models.__dict__[arch](num_classes, dropout, stride).cuda()

net = torch.nn.DataParallel(net, device_ids=list(range(args.ngpu)))

# optionally resume from a checkpoint
if os.path.isfile(args.resume):
    print_log("=> loading checkpoint '{}'".format(args.resume), log)
    checkpoint = torch.load(args.resume)
    recorder = checkpoint['recorder']
    args.start_epoch = checkpoint['epoch']
    net.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_acc = recorder.max_accuracy(False)
    print_log(
        "=> loaded checkpoint '{}' accuracy={} (epoch {})".format(
            args.resume, best_acc, checkpoint['epoch']), log)
else:
    print_log("=> no checkpoint found at '{}'".format(args.resume), log)

validate(test_loader, net, criterion, log)

In [ ]:
def validate(val_loader, model, log, eps=4, rand_init=False, mean=None, std=None):
    '''evaluate trained model'''
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    for i, (input, target) in enumerate(val_loader):
        if args.use_cuda:
            input = input.cuda()
            target = target.cuda()

        with torch.no_grad():
            input_var = Variable(input)
            target_var = Variable(target)

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

    if fgsm:
        print_log('Attack (eps : {}) Prec@1 {top1.avg:.2f}'.format(eps, top1=top1), log)
    else:
        print_log(
            '  **Test** Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f} Error@1 {error1:.3f} Loss: {losses.avg:.3f} '
            .format(top1=top1, top5=top5, error1=100 - top1.avg, losses=losses), log)
    return top1.avg, losses.avg